In [126]:
import pandas as pd
import numpy as np
import difflib
import googlemaps


residences = pd.DataFrame(columns=['address', 'zipcode', 'latitude', 'longitude'])

survivors = pd.DataFrame(columns=['name'])

contact_info = pd.DataFrame(columns=['survivor_id', 'contact_info'])

contact_points = pd.DataFrame(columns = ['residence_id', 'survivor_id', 'date', 
                                         'water_lvl', 'damage_lvl', 'notes1', 'notes2', 
                                         'notes3', 'intaker'])



In [128]:
keys = pd.read_csv('../data/API.txt')
gmap_key = keys.loc[keys['API']=='Google Maps', 'Key'].values[0]
gmaps = googlemaps.Client(key=gmap_key)

def find_or_add_residence(address):
    global residences
    try:
        geo = gmaps.geocode(address+', TN')
        lat = geo[0]['geometry']['location']['lat']
        lon = geo[0]['geometry']['location']['lng']
        zipcode = geo[0]['address_components'][-1]['long_name']

        coord_search = residences[(residences['latitude']==lat) & (residences['longitude']==lon)]

        if coord_search.shape[0] == 0:
            residences = residences.append({'address':address, 'latitude':lat, 
                                            'longitude':lon, 'zipcode':zipcode},
                                           ignore_index=True)
            res_id = residences.shape[0] - 1
            return res_id 
        else:
            res_id = coord_search.index.values[0]
            return res_id
    except:
        print(address)
        return address

In [129]:
def find_or_add_survivor(**kwargs):
    global survivors
    global contact_info
    
    name = kwargs['name'].upper().strip()
    name_search = survivors[survivors['name']==name]
    
    if name_search.shape[0] != 0:
        surv_id = name_search.index.values[0]
        
    elif len(difflib.get_close_matches(name, survivors['name'], n=1, cutoff=0.90)) > 0:
        match = difflib.get_close_matches(name, survivors['name'], n=1, cutoff=0.90)[0]
        choice = ''
        
        while choice == '':
        
            choice = input(f'You entered {name}. Did you mean: {match}? Type Y or N. \n')

            if choice in ['Y','y']:
                surv_id = survivors.loc[survivors['name']==match].index.values[0]

            elif choice in ['N','n']:
                survivors = survivors.append({'name':name},ignore_index=True)
                surv_id = residences.shape[0] - 1

            else:
                choice = ''    
    else:
        survivors = survivors.append({'name':name},ignore_index=True)
        surv_id = survivors.shape[0] - 1
    
    for k,v in kwargs.items():
        if k != 'name':
            contact_info = contact_info.append({'survivor_id':surv_id, 'contact_info':v}, ignore_index=True)
    
    return surv_id
        

In [130]:
def create_contact_point(**kwargs):
    global contact_points
    contact_points = contact_points.append(kwargs, ignore_index=True)

    

nash_codes = pd.read_csv('../data/CodesDamageAssessment_4_15_21.csv', parse_dates=['CreationDate'], 
                         dtype={'Address':'object','Extent of Damage':'object','Type of Structure':'object',
                                'Water Level':'float64','Note':'object','Creator':'object'})
nash_codes = nash_codes.dropna(subset=['Address'])
nash_codes = nash_codes.drop_duplicates(ignore_index=True)

mask = (nash_codes['Type of Structure']=='Residential') & (nash_codes['Extent of Damage'] != 'None')
for ind, row in nash_codes[mask].iterrows():
    add = row['Address'].upper().strip()
    res_id = find_or_add_residence(address=add)
    surv_id = None
    create_contact_point(residence_id = res_id, survivor_id = surv_id, 
                         date = row['CreationDate'], water_lvl = row['Water Level'], 
                         damage_lvl = row['Extent of Damage'], notes1=row['Notes'], 
                         intaker='Nashville Codes: ' + row['Creator'])

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info()

oem = pd.read_csv('../data/OEM_4_12_21.csv', 
                         dtype={'PropHouse':'object','PropZip':'object','PropStreet':'object',
                                'Extent of Damage':'object','Water Level':'object','Notes':'object',
                                'Damage 2':'object'})

oem = oem.fillna('')
oem = oem.drop_duplicates(ignore_index=True)

mask = oem['Extent of Damage'] != 'None'
for ind, row in oem[mask].iterrows():
    add = row['PropHouse'].upper().strip() + ' ' + row['PropStreet'].upper().strip()
    res_id = find_or_add_residence(address=add)
    surv_id = None
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-04-12'), 
                         water_lvl = row['Water Level'], damage_lvl = row['Extent of Damage'], 
                         notes1='NOTES: '+row['Notes'], notes2='DAMAGE 2: ' + row['Damage 2'], intaker='OEM')

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info()

nerve_4_9 = pd.read_csv('../data/NERVE_040921.csv', parse_dates=[10],
                         dtype={'Street No.':'object','Street':'object','First Name':'object','Last Name':'object',
                                'Email':'object','Phone Number':'object','Structure Damage Level':'object',
                                'Property Damage Described':'object','Comments':'object',
                                'Other - Property Damage Described':'object'})
nerve_4_9 = nerve_4_9.fillna('')
nerve_4_9 = nerve_4_9.drop_duplicates(ignore_index=True)

for ind, row in nerve_4_9.iterrows():
    add = row['Street No.'].upper().strip() + ' ' + row['Street'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['First Name'] + ' ' + row['Last Name'].upper().strip()
    
    if name == ' ':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone Number'].strip(), contact_2 = row['Email'].strip())
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = row['CreationDate'], 
                         damage_lvl = row['Structure Damage Level'], intaker='NERVE: Self Reported',
                         notes1='Property Damage Described: ' + row['Property Damage Described'], 
                         notes2 = 'Comments:' + row['Comments'], 
                         notes3 = 'Other - Property Damage Described: ' + row['Other - Property Damage Described'])

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info()

canvass_3_30 = pd.read_csv('../data/canvass_3_30.csv',
                         dtype={'Street Number':'object','Street Name':'object','Name':'object',
                                'Language Preference':'object','Saturday Help':'object',
                                'Phone Number':'object','Notes':'object'})
canvass_3_30 = canvass_3_30.fillna('')
canvass_3_30 = canvass_3_30.drop_duplicates(ignore_index=True)

for ind, row in canvass_3_30.iterrows():
    add = row['Street Number'].upper().strip() + ' ' + row['Street Name'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['Name']
    if name == '':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone Number'].strip())
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-03-31'),
                         intaker='Canvass Volunteer from Hands On Nashville',
                         notes1= 'Language Preference: ' + row['Language Preference'], 
                         notes2 = 'Notes' + row['Notes'], 
                         notes3 ='Followup: ' + row['Saturday Help'])

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info()

canvass_4_2 = pd.read_csv('../data/canvass_4_2.csv', parse_dates=[2],
                         dtype={'What is the house number?':'object','What is the street name?':'object',
                                'Is there visible debris on the property?':'object','Resident Name':'object',
                                'Resident phone number':'object','Language?':'object',
                                'Was the home impacted? (Either can you see this visually or did the resident disclose?)':'object',
                                'Does the resident want volunteers to assist with response?':'object',
                                'Does the resident need any of the following?':'object'})

canvass_4_2 = canvass_4_2.fillna('')
canvass_4_2 = canvass_4_2.drop_duplicates(ignore_index=True)


mask = canvass_4_2['Was the home impacted? (Either can you see this visually or did the resident disclose?)'] == 'Yes'
for ind, row in canvass_4_2[mask].iterrows():
    add = row['What is the house number?'].upper().strip() + ' ' + row['What is the street name?'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['Resident Name']
    if name == '':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Resident phone number'].strip())
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = row['Completion time'],
                         intaker='Canvass Volunteer from Hands On Nashville',
                         notes1= 'Language: ' + row['Language?'], 
                         notes2 = 'Requested Assistance: ' + row['Does the resident want volunteers to assist with response?'], 
                         notes3 ='Specific Needs: ' + row['Does the resident need any of the following?'])

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info()

nash_responds = pd.read_csv('../data/nash_responds_4_6.csv',
                         dtype={'First Name':'object','Last Name':'object','Address 1':'object',
                                'Email':'object','Phone':'object','Request Details':'object'})
nash_responds = nash_responds.drop_duplicates(ignore_index=True)



for ind, row in nash_responds.iterrows():
    add = row['Address 1'].upper().strip()
    res_id = find_or_add_residence(address=add)
    name = row['First Name'] + ' ' + row['Last Name']
    
    if name == ' ':
        surv_id = None
    else:
        surv_id = find_or_add_survivor(name = name, contact_1 = row['Phone'].strip(), contact_2 = row['Email'])
    
    create_contact_point(residence_id = res_id, survivor_id = surv_id, date = pd.to_datetime('2021-04-06'), 
                         intaker='Nashville Responds: Self Reported',
                         notes1='Request Details: ' + row['Request Details'])

print('CONTACT POINTS'), contact_points.info(), print('\nRESIDENCES'), residences.info(), print('\nSURVIVORS'), survivors.info(), print('\nCONTACT INFO'), contact_info.info()

survivors.to_csv('../data/survivors_table.csv')
residences.to_csv('../data/residences_table.csv')
contact_info.to_csv('../contact_info_table.csv')
contact_points.to_csv('../data/contact_points_table.csv')